## Objetivo do cralwer: Encontrar no site Airbnb, para uma determinada cidade, diversas informacoes a respeito de todas as acomodacoes atualmente disponíveis

## Cidade

In [1]:
# cidade desejada para buscar acomodações
cidade = 'Icapuí'

## Path do Driver

In [2]:
# path onde foi instalado o chromedriver
path_to_chromedriver = '/home/yan/Data-Science-Projects/chromedriver'

## Flag

In [3]:
flag_visual = 0

## Imports

In [4]:
import pandas as pd
import numpy as np
import unidecode
import pickle
import time
from selenium import webdriver
from joblib import Parallel, delayed

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 20)

## Metodos

In [5]:
# trata o nome da cidade de acordo com o padrao esperado pelo site
def trata_nome_cidade(cidade):
    return (unidecode.unidecode(cidade.lower().replace(' ', '_')))

# formata a url de acordo com os parametros passados
def prepara_cidade_url(main_url, cidade, preco1, preco):
    return (main_url.format(cidade, preco1, preco2))

# retorna uma lista de faixas de preco
def get_faixas_precos(lista_precos):
    faixas_precos = []
    for i in range(len(lista_precos) - 1):
        faixas_precos.append([lista_precos[i], lista_precos[i+1]-1])
    return (faixas_precos)

# mostra o tempo de execucao decorrido
def print_tempo_execucao(start_time, end_time):
    print ('Tempo de execução: {} segundos'.format(round((end_time - start_time), 2)))

# recebe uma url de acomodacao e retorna suas informacos
def extrai_infos_urls(url_casa):
    
    # inicializa o driver
    driver = webdriver.Chrome(path_to_chromedriver, options=chromeOptions)
    time.sleep(0.1)
    
    # inicializa flags
    achou_lugar = 0
    achou_preco = 0
    
    # entra na url da acomodação
    driver.get(url_casa)
    
    # tempo de carregar a página
    time.sleep(0.4)
    
    for i in range(5):
        if (achou_lugar == 1):
            break
        try:
            nome_lugar = driver.find_element_by_xpath(xpath_nome_lugar).text
            achou_lugar = 1
        except:
            try:
                nome_lugar = driver.find_elements_by_xpath(xpath_nome_lugar_2)[0].text
                achou_lugar = 1
            except:
                time.sleep(1)
                continue

            
    if not achou_lugar:
        return (missing_values)
    
    time.sleep(0.3)
    
    # tipo 1 da acomodação
    try:
        tipo1 = driver.find_elements_by_xpath(xpath_tipo_1)[0].text
    except:
        tipo1 = ''
        
    
    # tipo 2 da acomodação
    try:
        tipo2 = driver.find_elements_by_xpath(xpath_tipo_2)[0].text
    except:
        tipo2 = ''
    
    # capacidade da acomodação
    try:
        capacidade_casa = driver.find_elements_by_xpath(xpath_capacidade_casa)[1].text
    except:
        try:
            capacidade_casa = driver.find_elements_by_xpath('//div[@class="_tqmy57"]')[1].text
        except:
            capacidade_casa = ''

    # preço da diaria    
    for i in range(5):
        if (achou_preco == 1):
            break
        try:
            preco_diaria = driver.find_element_by_xpath(xpath_preco_diaria1).text
            achou_preco = 1
        except:
            try:
                preco_diaria = driver.find_element_by_xpath(xpath_preco_diaria2).text
                achou_preco = 1
            except:
                time.sleep(1)
                
    preco_diaria = str(preco_diaria).split('<')[0].replace('por noite', '').strip()

    # nome do anfitriao
    try:
        anfitriao_acomodacao = driver.find_element_by_xpath(xpath_anfitriao).text
    except:
        try:
            anfitriao_acomodacao = driver.find_element_by_xpath(xpath_anfitriao_2).text.replace('Hospedado por ', '')
        except:
            anfitriao_acomodacao = ''
    
    # regiao da acomodação
    try:
        regiao_acomodacao = driver.find_elements_by_xpath(xpath_regiao)[0].text
    except:
        try:
            regiao_acomodacao = driver.find_element_by_xpath(xpath_regiao_2).text
        except:
            regiao_acomodacao = ''
            
    time.sleep(0.3)
            
    # nota media da acomodacao
    try:
        nota = driver.find_elements_by_xpath(xpath_nota)[0].text
    except:
        try:
            nota = driver.find_elements_by_xpath(xpath_nota_2)[0].text
        except:
            try:
                nota = driver.find_elements_by_xpath(xpath_nota_3)[0].text
            except:
                nota = ''
                
                
    if nota == np.nan:
        nota = ''
    
    # numero de comentarios da acomodacao    
    try:
        comentarios = driver.find_elements_by_xpath(xpath_comentarios)[0].text
    except:
        try:
            comentarios = driver.find_elements_by_xpath(xpath_comentarios_2)[0].text
        except:
            comentarios = ''
            
    if '(' in comentarios:
        try:
            comentarios = comentarios.split('(')[1].split(' ')[0]
        except:
            comentarios = ''

    if comentarios == np.nan:
        comentarios = ''

            
    time.sleep(0.3)
    
    # número de quartos da comodação
    try:
        quartos_casa = driver.find_elements_by_xpath(xpath_quartos_casa)[2].text
    except:
        try:
            quartos_casa = driver.find_elements_by_xpath('//div[@class="_tqmy57"]')[1].text
        except:
            quartos_casa = ''
            
    # fecha o driver    
    driver.quit()
        
    # retorna resultados
    return ([cidade, anfitriao_acomodacao, url_casa, nome_lugar,
             regiao_acomodacao, tipo1, tipo2, capacidade_casa, quartos_casa,
             preco_diaria, nota, comentarios])

## Parametros globais

In [6]:
# url principal para pesquisar acomodacoes
main_url = 'https://www.airbnb.com.br/s/{0}/homes?refinement_paths%5B%5D=%2Fhomes&search_type=filter_change&price_min={1}&price_max={2}'

# condicao de parada do crawler
condicao_parada = 'items_offset=288'

# xpaths com os itens desejados
xpath_elements_pagina = '//a[contains(@href, "/rooms")]'
xpath_botao_muda_pagina = '//div[@class="_1m76pmy"]'
xpath_preco_diaria1 = '//span[@class="_doc79r"]'
xpath_preco_diaria2 = '//span[contains(text(), "R$")]'
xpath_nome_lugar = '//span[@class="_18hrqvin"]'
xpath_nome_lugar_2 = '//h1[@class="_14i3z6h"]'
xpath_quartos_casa = '//div[@class="_czm8crp"]'
xpath_capacidade_casa = '//div[@class="_czm8crp"]'
xpath_tipo_1 = '//span[@class="_1p3joamp"]'
xpath_tipo_2 = '//div[@class="_1p3joamp"]'
xpath_regiao = '//div[@class="_czm8crp"]'
xpath_regiao_2 = '//a[@class="_5twioja"]'
xpath_anfitriao = '//div[@class="_8b6uza1"]'
xpath_anfitriao_2 = '//h2[@class="_14i3z6h"]'
xpath_nota = '//div[@class="_14i3z6h"]'
xpath_nota_2 = '//div[@class="_10za72m2"]'
xpath_nota_3 = '//div[@class="_tghtxy2"]'
xpath_comentarios = '//span[@class="_1p0spma2"]'
xpath_comentarios_2 = '//span[@class="_1plk0jz1"]'

# lista de cidades
lista_colunas = ['cidade', 'anfitriao', 'url_acomodacao', 'nome_acomodacao', 'regiao',
'tipo1', 'tipo2', 'capacidade', 'quartos', 'preco', 'nota_media', 'num_comentarios']

# valores que identificam acomodação
lista_valores_acomodacao = ['Casa',
'casa',
'quarto',
'Quarto',
'acomodação',
'Acomodação',
'moradia',
'Moradia']

lista_colunas_ordenadas = ['cidade',
'anfitriao',
'url_acomodacao',
'nome_acomodacao',
'regiao',
'tipo1',
'tipo2',
'tipo_acomodacao',
'capacidade',
'quartos',
'preco',
'nota_media',
'num_comentarios']

# faixas de preco das acomodacoes
# obs: metodo utilizado para maximizar o numero de acomodacoes encontradas
lista_precos = [1, 30, 50, 60, 80, 90, 100, 101, 110, 120,
                150, 160, 170, 180, 190, 200, 210, 220, 230, 240,
                250, 260, 280, 300, 350, 400, 450, 500, 600,
                700, 800, 1000, 2000, 100000]

# numero de navegadores utilizados em paralelo
num_browsers = 5

missing_values = ['' for i in range(len(lista_colunas))]

## Setup

In [7]:
# trata nome da cidade para colocar no formado a ser colocado na url
cidade_tratada = trata_nome_cidade(cidade)
print ('Cidade:', cidade_tratada)

# determina faixas de precos (lista de valores min e max) de cada busca
faixas_precos = get_faixas_precos(lista_precos)

# parametros do webdriver
chromeOptions = webdriver.ChromeOptions()

if (flag_visual == 0):
    chromeOptions.add_argument("--headless")

Cidade: icapui


## Parte 1) Acha todas as acomodacoes da cidade

In [8]:
start_time = time.time()

# inicializa o driver
driver = webdriver.Chrome(path_to_chromedriver, options=chromeOptions)

lista_url_casas = []

for preco1, preco2 in faixas_precos:

    flag_parada = 0
    lista_urls_visitadas = []

    # prepara a url com base no nome da cidade
    url_atual = prepara_cidade_url(main_url, cidade_tratada, preco1, preco2)

    driver.get(url_atual)

    while(flag_parada == 0):
        time.sleep(0.5)
        # caso seja a ultima pagina da cidade
        url_atual = driver.current_url

        # caso ja tenha visitado essa pagina
        if (url_atual in lista_urls_visitadas):
            flag_parada = 1
            break

        # caso seja a ultima pagina
        if (condicao_parada in url_atual):
            flag_parada = 1

        # atualiza lista de urls visitadas
        lista_urls_visitadas.append(url_atual)

        # lista elementos da pagina
        time.sleep(1)
        list_elements = driver.find_elements_by_xpath(xpath_elements_pagina)
        time.sleep(1)


        # coloca na lista acomodacoes encontradas
        for element in list_elements:
            try:
                url_casa = element.get_attribute('href')
                if (url_casa not in lista_url_casas):
                    lista_url_casas.append(url_casa)
            except Exception as e:
                continue

        # muda de pagina
        if (flag_parada == 0):
            try:
                botao = driver.find_elements_by_xpath(xpath_botao_muda_pagina)[-1]
                driver.execute_script("arguments[0].click();", botao)
            except:
                pass

# fecha o driver    
driver.quit()

print ('Numero encontrado de acomodacoes:', len(lista_url_casas))

# salva resultado em um arquivo pickle
with open('{}.pkl'.format(cidade_tratada), 'wb') as handle:
    pickle.dump(lista_url_casas, handle, protocol=pickle.HIGHEST_PROTOCOL)

# tempo de execucao
print_tempo_execucao(start_time, time.time())

Numero encontrado de acomodacoes: 84
Tempo de execução: 193.59 segundos


## Parte 2) Encontra informacoes das acomodacoes

In [9]:
start_time = time.time()

# leitura da lista de urls de acomodacoes encontradas
with open('{}.pkl'.format(cidade_tratada), 'rb') as handle:
    lista_url_casas = pickle.load(handle)
    
    
# extrai resultados das url em série
# i = 0
# resultados = []
# for url in lista_url_casas:
#     resultados.append(extrai_infos_urls(url))
#     if (i % 100 == 0):
#         print (i)
#     i+=1

# extrai resultados das url em paralelo
resultados = Parallel(n_jobs=num_browsers)(delayed(extrai_infos_urls)(url) for url in lista_url_casas)

# salva tudo em unico dataframe
resultados = [valor for valor in resultados if valor[0] != '']
df_output = pd.DataFrame(resultados, columns=lista_colunas)

# tempo de execucao
print_tempo_execucao(start_time, time.time())

Tempo de execução: 555.18 segundos


## Tratamento dos dados

In [10]:
# Tratamento da coluna tipo
lista_tipos = []
for tipo_1, tipo_2 in zip(df_output['tipo1'].values.tolist(), df_output['tipo2'].values.tolist()):
    if tipo_1 == '' and tipo_2 == '':
        tipo_novo = ''
        
    elif tipo_1 != '' and tipo_2 == '':
        tipo_novo = tipo_1
        
    elif tipo_1 == '' and tipo_2 != '':
        tipo_novo = tipo_2
        
    elif any(substring in tipo_2 for substring in lista_valores_acomodacao):
        tipo_novo = tipo_2
        
    else:
        tipo_novo = tipo_1
        
    lista_tipos.append(tipo_novo)
    
df_output['tipo_acomodacao'] = lista_tipos

df_output = df_output[lista_colunas_ordenadas]

## Salva resultado em uma planilha

In [11]:
# salva resultado em um arquivo xlsx
df_output.to_excel('resultados_{}.xlsx'.format(cidade_tratada), index=False)

## Alguns dos resultados obtidos

In [12]:
df_output.head(100)

,cidade,anfitriao,url_acomodacao,nome_acomodacao,regiao,tipo1,tipo2,tipo_acomodacao,capacidade,quartos,preco,nota_media,num_comentarios
0,Icapuí,Valter,https://www.airbnb.com.br/rooms/39141026?locat...,"Chalé Serra Olho D'água, aconchego e encanto",Icapuí,Casa inteira,,Casa inteira,3 hóspedes,1 quarto,R$60,,
1,Icapuí,Valter,https://www.airbnb.com.br/rooms/39141026?locat...,"Chalé Serra Olho D'água, aconchego e encanto","Icapuí, Ceará, Brasil",Casa inteira,,Casa inteira,3 hóspedes,1 quarto,R$60,,
2,Icapuí,Carlos Alberto,https://www.airbnb.com.br/rooms/32003805?locat...,Hostel Recanto do Caluca Praia de Manibu,CE,Cancelamento gratuito por 48 horas,Quarto inteiro em suíte de hóspedes,Quarto inteiro em suíte de hóspedes,3 hóspedes,Estúdio,R$65,,
3,Icapuí,Rodrigo,https://www.airbnb.com.br/rooms/42531458?locat...,Casa-Chalé na Praia de Peroba,"Icapuí, Ceará, Brasil",Casa inteira,Leia mais sobre este espaço,Casa inteira,4 hóspedes,1 quarto,R$70,,
4,Icapuí,Carlos Alberto,https://www.airbnb.com.br/rooms/32003180?locat...,Hostel Recanto do Caluca Praia de Manibu.,Centro,Cancelamento gratuito por 48 horas,Quarto compartilhado em casa,Quarto compartilhado em casa,1 hóspede,1 quarto,R$80,,
5,Icapuí,Paolo Viglietti,https://www.airbnb.com.br/rooms/15176217?locat...,pousada sara,Icapuí,Café da Manhã,Quarto inteiro em apartamento,Quarto inteiro em apartamento,2 hóspedes,1 quarto,R$100,,
6,Icapuí,Francisco De Assis,https://www.airbnb.com.br/rooms/32585984?locat...,Chalé com vista panorâmica da Praia da Peroba,"Icapuí, Ceará, Brasil",Casa inteira,Leia mais sobre este espaço,Casa inteira,4 hóspedes,1 quarto,R$100,,
7,Icapuí,Rosina,https://www.airbnb.com.br/rooms/20332546?locat...,Quarto Carnaúba ou Almirante,Icapuí,Cancelamento gratuito por 48 horas,Quarto inteiro em casa,Quarto inteiro em casa,2 hóspedes,1 quarto,R$100,,
8,Icapuí,Paolo Viglietti,https://www.airbnb.com.br/rooms/15176217?locat...,pousada sara,Icapuí,Café da Manhã,Quarto inteiro em apartamento,Quarto inteiro em apartamento,,1 quarto,,,
9,Icapuí,Francisco De Assis,https://www.airbnb.com.br/rooms/32585984?locat...,Chalé com vista panorâmica da Praia da Peroba,Icapuí,Casa inteira,,Casa inteira,4 hóspedes,1 quarto,R$100,,
